<a href="https://colab.research.google.com/github/NMJ0/Detailed-Router/blob/main/detailed_router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required Installations and imports**

In [1]:
!pip install --break-system-packages https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/project/def/c17.def
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/lef/sky130.lef

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.9/629.9 kB 5.3 MB/s eta 0:00:00
--2025-05-19 04:34:04--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/project/def/c17.def
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15173 (15K) [text/plain]
Saving to: ‘c17.def’

c17.def             100%[===================>]  14.82K  --.-KB/s    in 0.001s  

2025-05-19 04:34:05 (10.8 MB/s) - ‘c17.def’ saved [15173/15173]

--2025-05-19 04:34:05--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/lef/sky130.lef
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:4

In [14]:
import requests
import LEFDEFParser
from LEFDEFParser import Rect
import math
import heapq as hq


**Parser for global routing solutions**.\
Returns a dictionary with net names as keys and their global routing solutions as values

In [3]:
url = "https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/gr/c17.guide"
response = requests.get(url)
lines = response.text.splitlines()
global_route={}
n=len(lines)
m=0
while True:
  if m>=n-1:
    break
  name=lines[m]
  global_route[name]=[]
  m+=1
  while True:
    if lines[m]==')':
      m+=1
      break
    if lines[m]=='(':
      m+=1
    x1,y1,x2,y2,layer=map(str,lines[m].split())
    global_route[name].append([layer,[int(x1),int(y1),int(x2),int(y2)]])
    m+=1
print(global_route)
#global_route

{'N1': [['li1', [13800, 20700, 20700, 27600]], ['met1', [13800, 20700, 20700, 27600]]], 'N1_d': [['met2', [0, 20700, 6900, 34500]], ['met1', [0, 27600, 6900, 34500]], ['met1', [0, 27600, 13800, 34500]], ['li1', [6900, 27600, 13800, 34500]], ['met2', [0, 20700, 6900, 41420]]], 'N2': [['li1', [13800, 20700, 20700, 27600]], ['met1', [13800, 20700, 20700, 27600]]], 'N22': [['li1', [6900, 6900, 13800, 13800]], ['met1', [6900, 6900, 13800, 13800]], ['met2', [6900, 0, 13800, 13800]]], 'N22_d': [['li1', [13800, 20700, 20700, 27600]], ['met1', [13800, 20700, 20700, 27600]], ['met2', [13800, 6900, 20700, 27600]], ['met1', [13800, 6900, 20700, 13800]], ['li1', [13800, 6900, 20700, 13800]]], 'N23': [['li1', [20700, 6900, 30700, 13800]], ['met1', [20700, 6900, 30700, 13800]], ['met2', [20700, 0, 30700, 13800]]], 'N23_d': [['li1', [13800, 13800, 20700, 20700]], ['met1', [13800, 13800, 20700, 20700]], ['met2', [13800, 13800, 20700, 27600]], ['met1', [13800, 20700, 20700, 27600]], ['met1', [13800, 207

**Skip Cells and Skip Nets**

In [4]:
skipCells = {"sky130_fd_sc_hd__decap_3", "sky130_fd_sc_hd__decap_4", "sky130_fd_sc_hd__decap_6", "sky130_fd_sc_hd__decap_8",\
            "sky130_fd_sc_hd__decap_12", "sky130_fd_sc_hd__fill_1", "sky130_fd_sc_hd__fill_2", "sky130_fd_sc_hd__fill_4",
            "sky130_fd_sc_hd__fill_8", "sky130_fd_sc_hd__lpflow_decapkapwr_3", "sky130_fd_sc_hd__lpflow_decapkapwr_4",\
            "sky130_fd_sc_hd__lpflow_decapkapwr_6", "sky130_fd_sc_hd__lpflow_decapkapwr_8", "sky130_fd_sc_hd__lpflow_decapkapwr_12", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_1", "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_2", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_hl_isowell_tap_4", "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_1", \
            "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_2", "sky130_fd_sc_hd__lpflow_lsbuf_lh_isowell_tap_4", "sky130_fd_sc_hd__tap_1", \
            "sky130_fd_sc_hd__tap_2", "sky130_fd_sc_hd__tapvgnd2_1", "sky130_fd_sc_hd__tapvgnd_1", \
            "sky130_fd_sc_hd__tapvpwrvgnd_1", "sky130_ef_sc_hd__decap_12"}

layerColors = { 'li1': 'red', 'met1': 'blue', 'met2': 'green', 'met3': 'orange', 'met4': 'magenta', 'met5': 'cyan' }


# skip power/ground/clock nets
skipNets = {'clk', 'VPWR', 'VGND'}



**DEF and LEF reader. Reading cells and nets.**

In [5]:
l = LEFDEFParser.LEFReader()
leffile="sky130.lef"
l.readLEF(leffile)
d = LEFDEFParser.DEFReader()
deffile="c17.def"
d.readDEF(deffile)

macros={}
for i in l.macros():
  macros[i.name()]=i

cells={}
for i in d.components():
  cells[i]=i.macro()

nets={}
for i in d.nets():
  nets[i.name()]=i.pins()
print(macros)
print(cells)
print(nets)
#cells
#nets


{'sky130_fd_sc_hd__a2bb2o_1': <LEFDEFParser.Macro object at 0x7b43caf2d1f0>, 'sky130_fd_sc_hd__a2bb2o_2': <LEFDEFParser.Macro object at 0x7b43caf2cd30>, 'sky130_fd_sc_hd__a2bb2o_4': <LEFDEFParser.Macro object at 0x7b43caf2d070>, 'sky130_fd_sc_hd__a2bb2oi_1': <LEFDEFParser.Macro object at 0x7b43caf2d170>, 'sky130_fd_sc_hd__a2bb2oi_2': <LEFDEFParser.Macro object at 0x7b43caf2d2b0>, 'sky130_fd_sc_hd__a2bb2oi_4': <LEFDEFParser.Macro object at 0x7b43caf2d2f0>, 'sky130_fd_sc_hd__a21bo_1': <LEFDEFParser.Macro object at 0x7b43caf2d330>, 'sky130_fd_sc_hd__a21bo_2': <LEFDEFParser.Macro object at 0x7b43caf2d370>, 'sky130_fd_sc_hd__a21bo_4': <LEFDEFParser.Macro object at 0x7b43caf2d3b0>, 'sky130_fd_sc_hd__a21boi_0': <LEFDEFParser.Macro object at 0x7b43caf2d3f0>, 'sky130_fd_sc_hd__a21boi_1': <LEFDEFParser.Macro object at 0x7b43caf2d430>, 'sky130_fd_sc_hd__a21boi_2': <LEFDEFParser.Macro object at 0x7b43caf2d470>, 'sky130_fd_sc_hd__a21boi_4': <LEFDEFParser.Macro object at 0x7b43caf2d4b0>, 'sky130_fd_

**Extracting track information, layer width and spacing.**

In [114]:
layerWidth = {}
layerSpacing = {}
for layer in l.layers():
    layerWidth[layer.name()] = layer.width()
    layerSpacing[layer.name()] = layer.pitch() - layer.width()
tracks=d.tracks()
tracks_ = {
    'li1':  [tracks['li1'][0].x,  tracks['li1'][0].num,  tracks['li1'][0].step],
    'met1': [tracks['met1'][1].x, tracks['met1'][1].num, tracks['met1'][1].step],
    'met2': [tracks['met2'][0].x, tracks['met2'][0].num, tracks['met2'][0].step],
    'met3': [tracks['met3'][1].x, tracks['met3'][1].num, tracks['met3'][1].step],
    'met4': [tracks['met4'][0].x, tracks['met4'][0].num, tracks['met4'][0].step],
    'met5': [tracks['met5'][1].x, tracks['met5'][1].num, tracks['met5'][1].step],
}
print(tracks_)
print(layerWidth)
print(layerSpacing)

{'li1': [230, 67, 460], 'met1': [170, 122, 340], 'met2': [230, 67, 460], 'met3': [340, 61, 680], 'met4': [460, 33, 920], 'met5': [1700, 12, 3400]}
{'li1': 170, 'met1': 140, 'met2': 140, 'met3': 300, 'met4': 300, 'met5': 1600}
{'li1': 170, 'met1': 200, 'met2': 320, 'met3': 380, 'met4': 620, 'met5': 1800}


In [113]:
def bloat(r, s):
  return Rect(r.ll.x - s, r.ll.y - s, r.ur.x + s, r.ur.y + s)

**Transforming pin locations  and obstructions acccording to cell orientation and storing it in a dictionary**

In [117]:
class Inst:
  def __init__(self, comp, macro):
    self._comp = comp
    self._macro = macro
    origin = comp.location()
    self._bbox = Rect(origin.x, origin.y, origin.x + macro.xdim(), origin.y + macro.ydim())
    self._pins = dict()
    self._obsts = dict()
    for p in macro.pins():
      shapes = dict()
      for port in p.ports():
        for layer, rects in port.items():
          if layer not in layerColors: continue
          if layer not in shapes: shapes[layer] = list()
          for v in rects:
            r = Rect(v.ll.x, v.ll.y, v.ur.x, v.ur.y)
            r.transform(comp.orient(), origin, macro.xdim(), macro.ydim())
            shapes[layer].append(r)
      self._pins[p.name()] = shapes

    for layer, rects in macro.obstructions().items():
      if layer not in layerColors: continue
      if layer not in self._obsts: self._obsts[layer] = list()
      for v in rects:
        r = Rect(v.ll.x, v.ll.y, v.ur.x, v.ur.y)
        s=layerSpacing[layer]
        r=bloat(r,s)
        r.transform(comp.orient(), origin, macro.xdim(), macro.ydim())
        self._obsts[layer].append(r)
cell_pin={}
obstructions={'li1':[],'met1':[],'met2':[],'met3':[],'met4':[],'met5':[]}
for r in cells.keys():
  inst=Inst(r,macros[cells[r]])
  for j in inst._obsts.keys():
    obstructions[j].extend(inst._obsts[j])
  for i in inst._pins.keys():
    for k in inst._pins[i].keys():
      obstructions[k].extend(inst._pins[i][k])
    if len(inst._pins[i])==0:
      continue
    cell_pin[(inst._comp.name(),i)]=inst._pins[i]
print(cell_pin)
print(obstructions)


{('PHY_EDGE_ROW_0_Left_7', 'VGND'): {'met1': [[(5520,10640),(6900,11120)]]}, ('PHY_EDGE_ROW_0_Left_7', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VGND'): {'met1': [[(23460,10640),(24840,11120)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VPWR'): {'met1': [[(5520,18800),(6900,19280)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VPWR'): {'met1': [[(23460,18800),(24840,19280)]]}, ('PHY_EDGE_ROW_3_Left_10', 'VGND'): {'met1': [[(5520,21520),(6900,220

**Adding boundary pins**

In [7]:
b_pins={}
for i in d.pins():
  b_pins['PIN',i.name()]=i.ports()[0]

for i in b_pins:
  cell_pin[i]=b_pins[i]
  for j in b_pins[i].keys():
    obstructions[j].extend(b_pins[i][j])
print(cell_pin)
print(obstructions)

{('PHY_EDGE_ROW_0_Left_7', 'VGND'): {'met1': [[(5520,10640),(6900,11120)]]}, ('PHY_EDGE_ROW_0_Left_7', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VGND'): {'met1': [[(23460,10640),(24840,11120)]]}, ('PHY_EDGE_ROW_0_Right_0', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_1_Left_8', 'VPWR'): {'met1': [[(5520,13360),(6900,13840)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_1_Right_1', 'VPWR'): {'met1': [[(23460,13360),(24840,13840)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VGND'): {'met1': [[(5520,16080),(6900,16560)]]}, ('PHY_EDGE_ROW_2_Left_9', 'VPWR'): {'met1': [[(5520,18800),(6900,19280)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VGND'): {'met1': [[(23460,16080),(24840,16560)]]}, ('PHY_EDGE_ROW_2_Right_2', 'VPWR'): {'met1': [[(23460,18800),(24840,19280)]]}, ('PHY_EDGE_ROW_3_Left_10', 'VGND'): {'met1': [[(5520,21520),(6900,220

**Converting each net into rectangular regions that are to be connected .**

In [9]:
nets_2={}
for i in nets.keys():
  if i in skipNets:
    continue
  nets_2[i]=[]
  for j in nets[i]:
    nets_2[i].append(cell_pin[j])
print(nets_2)

{'N1': [{'li1': [[(15165,24780),(15495,25625)], [(15165,25625),(15555,25705)], [(15340,25705),(15555,25755)], [(15385,25755),(15555,26335)], [(15330,26335),(15555,26375)], [(15175,26375),(15555,26460)], [(15175,26460),(15505,26895)]]}, {'li1': [[(16465,25855),(16875,26160)]]}], 'N1_d': [{'met2': [[(5150,37420),(5430,41420)]]}, {'li1': [[(7005,28185),(7335,28555)]]}], 'N2': [{'li1': [[(17925,23335),(18255,24180)], [(17925,23255),(18315,23335)], [(18100,23205),(18315,23255)], [(18145,22625),(18315,23205)], [(18090,22585),(18315,22625)], [(17935,22500),(18315,22585)], [(17935,22065),(18265,22500)]]}, {'li1': [[(18945,23035),(19175,23375)], [(18945,22835),(19275,23035)]]}, {'li1': [[(18285,25925),(18775,26125)]]}], 'N22': [{'met2': [[(7450,0),(7730,4000)]]}, {'li1': [[(7905,12440),(8175,13345)], [(7905,11640),(8085,12440)], [(7905,11135),(8165,11640)]]}], 'N22_d': [{'li1': [[(15830,11595),(16110,12545)]]}, {'li1': [[(15725,24735),(15985,25415)], [(15725,25415),(15895,26615)], [(15725,26615

**Function to find intersection of two reqions**

In [105]:
def inter_section(r1,r2,adjLayer):
  layer1=r1[0]
  layer2=r2[0]
  if layer2 not in adjLayer[layer1]:
    return None
  x1 = max(r1[1][0], r2[1][0])
  y1 = max(r1[1][1], r2[1][1])
  x2 = min(r1[1][2], r2[1][2])
  y2 = min(r1[1][3], r2[1][3])
  if x1 <= x2 and y1 <= y2:
    return [layer1, layer2, (x1, y1, x2, y2)]
  else :
    return None



**Function to convert nets into graphs**

In [110]:
def net_to_graph(net,global_solution,obstructions):
  layerOrient = { 'li1': 'VERTICAL', 'met1': 'HORIZONTAL', 'met2': 'VERTICAL', 'met3': 'HORIZONTAL', 'met4': 'VERTICAL', 'met5': 'HORIZONTAL' }
  adjLayer = {
  'li1':  ['met1'],
  'met1': ['li1',  'met2'],
  'met2': ['met1', 'met3'],
  'met3': ['met2', 'met4'],
  'met4': ['met3', 'met5'],
  'met5': ['met4']
  }
  graph=[]
  edges=[]
  weights={}
  for i in global_solution:
    layer,x1,y1,x2,y2=i[0],i[1][0],i[1][1],i[1][2],i[1][3]

    if layerOrient[layer]=='VERTICAL':
      n=tracks_[layer][2]
      x1-=tracks_[layer][0]
      x2-=tracks_[layer][0]
      start = ((x1 + n - 1) // n) * n
      lx=[i+tracks_[layer][0] for i in range(start, x2 + 1, n)]
      ly=[]

      for j in adjLayer[layer]:
        n=tracks_[j][2]
        y1-=tracks_[j][0]
        y2-=tracks_[j][0]
        start = ((y1 + n - 1) // n) * n

        ly=[i+tracks_[j][0] for i in range(start, y2 + 1, n)]
        for p in range(len(lx)):
          flag=0
          for q in range(len(ly)):
            graph.append((layer,(lx[p],ly[q])))
            if flag==1:
              edges.append(((layer,(lx[p],ly[q-1])),(layer,(lx[p],ly[q]))))
            flag=1

    else:
      n=tracks_[layer][2]
      y1-=tracks_[layer][0]
      y2-=tracks_[layer][0]
      start = ((y1 + n - 1) // n) * n
      ly=[i+tracks_[layer][0] for i in range(start, y2 + 1, n)]
      lx=[]
      for j in adjLayer[layer]:
        n=tracks_[j][2]
        x1-=tracks_[j][0]
        x2-=tracks_[j][0]
        start = ((x1 + n - 1) // n) * n
        lx=[i+tracks_[j][0] for i in range(start, x2 + 1, n)]
        for p in range(len(ly)):
          flag=0
          for q in range(len(lx)):
            graph.append((layer,(lx[q],ly[p])))
            if flag==1:
              edges.append(((layer,(lx[q-1],ly[p])),(layer,(lx[q],ly[p]))))
            flag=1
  graph_new=[]
  for i in graph:
    if i not in graph_new:
      graph_new.append(i)
  graph=graph_new
  for i in edges:

    weights[i]=1


  for i in range(len(global_solution)):
    for j in range(i+1,len(global_solution)):
      intersection=inter_section(global_solution[i],global_solution[j],adjLayer)
      if intersection!=None:
        layer1=intersection[0]
        layer2=intersection[1]
        x1=intersection[2][0]
        y1=intersection[2][1]
        x2=intersection[2][2]
        y2=intersection[2][3]
        for k in graph:
          if k[0] not in (layer1,layer2):
            continue
          if k[0]==layer1:
            layer=layer1
            layeradj=layer2
          else:
            layer=layer2
            layeradj=layer1
          x=k[1][0]
          y=k[1][1]
          if x1<=x<=x2 and y1<=y<=y2:
            edges.append(((layer1,(x,y)),(layer2,(x,y))))
            weights[((layer1,(x,y)),(layer2,(x,y)))]=3
  obs=[]
  for i in obstructions.keys():
    layer=i
    for j in obstructions[i]:
      x1=j.ll.x
      y1=j.ll.y
      x2=j.ur.x
      y2=j.ur.y
      for node in graph:
        if node[0]!=layer:
            continue
        x,y=node[1][0],node[1][1]
        if x1<=x<=x2 and y1<=y<=y2:
          obs.append(node)

  s_t=[]
  for i in net:
    p=[]
    for j in i.keys():
      layer=j
      for k in i[j]:
        x1=k.ll.x
        y1=k.ll.y
        x2=k.ur.x
        y2=k.ur.y
        for node in graph:
          if node[0]!=layer:
            continue
          x,y=node[1][0],node[1][1]
          if x1<=x<=x2 and y1<=y<=y2:
            if node in obs:
              obs.remove(node)

            p.append(node)

      for h in range(len(p)):
        for k in range(h+1,len(p)):

          edges.append((p[h],p[k]))
          weights[(p[h],p[k])]=0
    s_t.append(p[0])




  #print(graph)
  #print(edges)
  #print(weights)
  edges=weights.keys()
  for (u,v) in edges:
    if u not in graph:
      graph.append(u)
    if v not in graph:
      graph.append(v)
  adj = {v: [] for v in graph}
  for (u, v) in edges:
    adj[u].append(v)
    adj[v].append(u)

  return graph,weights,obs,s_t,adj









**Vertex class for Astar Algorithm .**

In [101]:
class Vertex:
  def __init__(self, node, cost=math.inf, parent=None, nbrs=None):
    self._node=node
    self._xy = node[1]

    self._g=0
    self._h=0
    self._cost = self._g+self._h
    self._parent = parent
    self._nbrs = nbrs
  def __lt__(self, r):
    return self._cost < r._cost
  def __eq__(self, r):
    return self._node == r._node
  def __repr__(self):
    return f'(xy:{self._node}, cost:{self._cost})'

**Priority Queue**

In [16]:
class priority_queue:
  def __init__(self, vertices = []):
    self._vertices = vertices[:]
    self._q = vertices[:]
    hq.heapify(self._q)
  def push(self, v):
    hq.heappush(self._q, v)
  def pop(self):
    return(hq.heappop(self._q))
  def update(self, v, cost):
    try: i = self._q.index(v)
    except ValueError: i = None
    if i is not None:
      self._q[i]._cost = cost
      hq.heapify(self._q)
  def updateIndex(self, i, cost):
    assert i < len(self._q)
    self._vertices[i]._cost = cost
    hq.heapify(self._q)
  def empty(self):
    return len(self._q) == 0
  def __contains__(self, v):
    return v in self._q
  def __repr__(self):
    return str(self._q)

In [18]:
def dist(u, v):
  return abs(u._xy[0] - v._xy[0]) + abs(u._xy[1] - v._xy[1])

**A-star Algorithm**

In [128]:
def astar(V, s, t,obs,weights):
  for v in V:
    if v._node in obs:
      v._g,v._h, v._parent,v._cost = math.inf,5*dist(v,t), None,math.inf

    else:
      v._g,v._h, v._parent,v._cost = math.inf,dist(v,t), None,math.inf


  s._g = 0
  s._h= dist(s,t)
  s._cost= s._g + s._h

  Q = priority_queue(V)
  #print(s._node,s._g,s._h)


  while not Q.empty():
    u = Q.pop()
    #print(u)
    if u == t: break
    for v in u._nbrs:
      if u == v: # Check if neighbor is the current node
        #print(u._node)
        w = 0
      else:
        try :
          w=weights[(u._node,v._node)]
        except :
          w=weights[(v._node,u._node)]
      if v._g > u._g + w:
        v._g = u._g + w
        v._parent = u
        if v in Q:
          Q.update(v, v._g + v._h)
        else:

          Q.push(v)

  path = [t]
  while path[-1]._parent is not None:
    path.append(path[-1]._parent)
  return path

**Using A star on the graph.**

In [124]:
net_name={}
for i in d.nets():
  net_name[i.name()]=i


In [131]:
for i in nets_2.keys():
  layerOrient = { 'li1': 'VERTICAL', 'met1': 'HORIZONTAL', 'met2': 'VERTICAL', 'met3': 'HORIZONTAL', 'met4': 'VERTICAL', 'met5': 'HORIZONTAL' }
  net=nets_2[i]
  name=i
  global_solution=global_route[i]
  graph,weights,obs,s_t,adj=net_to_graph(net,global_solution,obstructions)
  indx={}

  vertices=[Vertex(j) for j in graph]
  for j in vertices:
    j._nbrs=[]
    for k in adj[j._node]:
      j._nbrs.append(vertices[vertices.index(Vertex(k))])
  source=vertices[vertices.index(Vertex(s_t[0]))]
  print(s_t)
  new_obs=[]
  for j in range(1,len(s_t)):
    target=vertices[vertices.index(Vertex(s_t[j]))]
    path=astar(vertices,source,target,obs,weights)
    current_net=net_name[name]
    for k in range(len(path)-1):
      layer=path[k]._node[0]
      x=path[k]._xy[0]
      y=path[k]._xy[1]
      layer_=path[k+1]._node[0]
      x_=path[k+1]._xy[0]
      y_=path[k+1]._xy[1]
      if layer==layer_:
        width=layerWidth[layer]
        if layerOrient[layer]=='VERTICAL':
          y1=min(y,y_)
          y2=max(y,y_)
          if x==x_:
            r=Rect(x-width//2,y1,x_+width//2,y2)
            s=layerSpacing[layer]
            r=bloat(r,s)
            obstructions[layer].append(r)
            new_obs.append(r)

            current_net.addRect(layer,x-width//2,y1,x_+width//2,y2)
        else:
          x1=min(x,x_)
          x2=max(x,x_)
          if y==y_:
            r=Rect(x1,y-width//2,x2,y_+width//2)
            s=layerSpacing[layer]
            r=bloat(r,s)
            obstructions[layer].append(r)
            new_obs.append(r)
            current_net.addRect(layer,x1,y-width//2,x2,y_+width//2)
    #obstructions.extend(new_obs)



    print(path)


[('li1', (15410, 24990)), ('li1', (16790, 26010))]
[(xy:('li1', (16790, 26010)), cost:11), (xy:('li1', (16790, 26350)), cost:1710), (xy:('li1', (16790, 26690)), cost:3409), (xy:('met1', (16790, 26690)), cost:686), (xy:('met1', (16330, 26690)), cost:1145), (xy:('met1', (15870, 26690)), cost:1604), (xy:('met1', (15410, 26690)), cost:2063), (xy:('li1', (15410, 26690)), cost:2060), (xy:('li1', (15410, 24990)), cost:2400)]
[('met2', (5290, 37570)), ('li1', (7130, 28390))]
[(xy:('li1', (7130, 28390)), cost:53), (xy:('li1', (7130, 28050)), cost:1752), (xy:('met1', (7130, 28050)), cost:389), (xy:('met1', (6670, 28050)), cost:848), (xy:('met1', (6210, 28050)), cost:1307), (xy:('met2', (6210, 28050)), cost:1304), (xy:('met2', (6210, 28390)), cost:963), (xy:('met2', (6210, 28730)), cost:1302), (xy:('met2', (6210, 29070)), cost:1641), (xy:('met2', (6210, 29410)), cost:1980), (xy:('met2', (6210, 29750)), cost:2319), (xy:('met2', (6210, 30090)), cost:2658), (xy:('met2', (6210, 30430)), cost:2997), (

In [133]:
!pip install rtree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 7.1 MB/s eta 0:00:00


In [132]:
d.writeDEF('c17_out.def')


In [143]:
url = "https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/def/c17.def"
df = requests.get(url)

In [144]:
url = "https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/lef/sky130.lef"
lf = requests.get(url)

In [135]:
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/sol/checker.py

--2025-05-19 12:32:51--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/refs/heads/master/project/sol/checker.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16180 (16K) [text/plain]
Saving to: ‘checker.py’

checker.py          100%[===================>]  15.80K  --.-KB/s    in 0.001s  

2025-05-19 12:32:51 (12.9 MB/s) - ‘checker.py’ saved [16180/16180]



In [146]:
!python3 checker.py -i df -o c17_out.def -l lf

error: Cannot open LEF file lf
error: Cannot open DEF file df
Total number of spacing violations : 0
Total number of nets : 0
Total number of open nets : 0


In [106]:
def net_to_graph(net,global_solution,obstructions):
  layerOrient = { 'li1': 'VERTICAL', 'met1': 'HORIZONTAL', 'met2': 'VERTICAL', 'met3': 'HORIZONTAL', 'met4': 'VERTICAL', 'met5': 'HORIZONTAL' }
  adjLayer = {
  'li1':  ['met1'],
  'met1': ['li1',  'met2'],
  'met2': ['met1', 'met3'],
  'met3': ['met2', 'met4'],
  'met4': ['met3', 'met5'],
  'met5': ['met4']
  }
  graph=[]
  edges=[]
  weights={}
  for i in global_solution:
    layer,x1,y1,x2,y2=i[0],i[1][0],i[1][1],i[1][2],i[1][3]

    if layerOrient[layer]=='VERTICAL':
      n=tracks_[layer][2]
      # Align coordinates to the track grid
      start_x = tracks_[layer][0] + math.ceil((x1 - tracks_[layer][0]) / n) * n
      end_x = tracks_[layer][0] + math.floor((x2 - tracks_[layer][0]) / n) * n
      lx=[i for i in range(start_x, end_x + 1, n)]

      for j in adjLayer[layer]:
        n_adj=tracks_[j][2]
        # Align coordinates to the adjacent layer's track grid
        start_y = tracks_[j][0] + math.ceil((y1 - tracks_[j][0]) / n_adj) * n_adj
        end_y = tracks_[j][0] + math.floor((y2 - tracks_[j][0]) / n_adj) * n_adj

        ly=[i for i in range(start_y, end_y + 1, n_adj)]
        for p in range(len(lx)):
          flag=0
          for q in range(len(ly)):
            graph.append((layer,(lx[p],ly[q])))
            if flag==1:
              edges.append(((layer,(lx[p],ly[q-1])),(layer,(lx[p],ly[q]))))
            flag=1

    else:
      n=tracks_[layer][2]
      # Align coordinates to the track grid
      start_y = tracks_[layer][0] + math.ceil((y1 - tracks_[layer][0]) / n) * n
      end_y = tracks_[layer][0] + math.floor((y2 - tracks_[layer][0]) / n) * n
      ly=[i for i in range(start_y, end_y + 1, n)]

      for j in adjLayer[layer]:
        n_adj=tracks_[j][2]
         # Align coordinates to the adjacent layer's track grid
        start_x = tracks_[j][0] + math.ceil((x1 - tracks_[j][0]) / n_adj) * n_adj
        end_x = tracks_[j][0] + math.floor((x2 - tracks_[j][0]) / n_adj) * n_adj
        lx=[i for i in range(start_x, end_x + 1, n_adj)]
        for p in range(len(ly)):
          flag=0
          for q in range(len(lx)):
            graph.append((layer,(lx[q],ly[p])))
            if flag==1:
              edges.append(((layer,(lx[q-1],ly[p])),(layer,(lx[q],ly[p]))))
            flag=1

  graph_new=[]
  for i in graph:
    if i not in graph_new:
      graph_new.append(i)
  graph=graph_new

  for i in edges:

    weights[i]=1

  for i in range(len(global_solution)):
    for j in range(i+1,len(global_solution)):
      intersection=inter_section(global_solution[i],global_solution[j],adjLayer)
      if intersection!=None:
        layer1=intersection[0]
        layer2=intersection[1]
        x1=intersection[2][0]
        y1=intersection[2][1]
        x2=intersection[2][2]
        y2=intersection[2][3]
        for k in graph:
          if k[0] not in (layer1,layer2):
            continue
          if k[0]==layer1:
            layer=layer1
            #layeradj=layer2 # This variable is not used
          else:
            layer=layer2
            #layeradj=layer1 # This variable is not used
          x=k[1][0]
          y=k[1][1]
          if x1<=x<=x2 and y1<=y<=y2:
            edges.append(((layer1,(x,y)),(layer2,(x,y))))
            weights[((layer1,(x,y)),(layer2,(x,y)))]=3

  # Add nodes from edges to graph if they are not already present
  for u, v in edges:
      if u not in graph:
          graph.append(u)
      if v not in graph:
          graph.append(v)

  obs=[]
  for i in obstructions.keys():
    layer=i
    for j in obstructions[i]:
      x1=j.ll.x
      y1=j.ll.y
      x2=j.ur.x
      y2=j.ur.y
      for node in graph:
        if node[0]!=layer:
            continue
        x,y=node[1][0],node[1][1]
        if x1<=x<=x2 and y1<=y<=y2:
          obs.append(node)

  s_t=[]
  for i in net:
    p=[]
    for j in i.keys():
      layer=j
      for k in i[j]:
        x1=k.ll.x
        y1=k.ll.y
        x2=k.ur.x
        y2=k.ur.y
        for node in graph:
          if node[0]!=layer:
            continue
          x,y=node[1][0],node[1][1]
          if x1<=x<=x2 and y1<=y<=y2:
            if node in obs:
              obs.remove(node)

            p.append(node)

      for h in range(len(p)):
        for k in range(h+1,len(p)):

          edges.append((p[h],p[k]))
          weights[(p[h],p[k])]=0
    if len(p) > 0: # Ensure that p is not empty before accessing index 0
      s_t.append(p[0])




  #print(graph)
  #print(edges)
  #print(weights)
  edges=weights.keys() # Re-get keys after adding weights
  adj = {v: [] for v in graph}
  for (u, v) in edges:
    adj[u].append(v)
    adj[v].append(u)

  return graph,weights,obs,s_t,adj